In [108]:
import numpy as np
import pandas as pd

In [109]:
clean = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/clean.csv")

In [110]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [111]:
clean  = fun.delete_id_columns(clean) 
clean = fun.replace_NAN_with_na(clean)
clean = fun.entry_to_lowercase(clean)
clean = fun.remove_underscores_spaces(clean)
clean = fun.impute_data(clean)
clean = fun.convert_to_categorical(clean)
clean

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,YEAR,Country,HHsizemembers,HHsizeMAE,HouseholdType,Head_EducationLevel,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,2.0,groundnut,na,na,2015,tanzania,4,3.32,single,primary,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,2.0,millet,na,na,2015,tanzania,10,5.85,together,primary,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,2.0,groundnut,na,na,2015,tanzania,5,4.32,together,illiterate,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,2.0,cowpea,na,na,2015,tanzania,11,9.33,together,primary,...,0.000000,0.000000,1.000000,0.310907,257.234727,257.234727,0,0.999984,0.000016,africa
4,2.0,sesame,na,na,2015,tanzania,6,5.41,together,secondary,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11957,3.0,maize,goodharvest,monoculture,2018,ghana,8,5.43,couple,primary,...,655.292919,0.000000,1.000000,0.779661,1615.869392,339.743848,7,0.629176,0.370824,africa
11958,5.0,maize,goodharvest,monoculture,2018,ghana,7,4.78,couple,noschool,...,0.000000,0.000000,1.000000,0.310907,355.648536,355.648536,5,1.000000,0.000000,africa
11959,2.0,maize,badharvest,intercrop,2018,ghana,4,3.20,mansingle,noschool,...,49.859244,0.000000,0.456522,0.456522,379.050204,276.284247,0,0.000000,1.000000,africa
11960,2.0,maize,badharvest,intercrop,2018,ghana,4,3.10,couple,noschool,...,0.000000,0.000000,1.000000,0.310907,693.504198,693.504198,0,1.000000,0.000000,africa


In [92]:
clean.dtypes

crop_count                                       float64
crop_name_1                                     category
crop_harvest_1                                  category
crop_intercrop_1                                category
YEAR                                               int64
Country                                         category
HHsizemembers                                      int64
HHsizeMAE                                        float64
HouseholdType                                   category
Head_EducationLevel                             category
LandOwned                                        float64
LandCultivated                                   float64
LivestockHoldings                                float64
NrofMonthsFoodInsecure                             int64
PPI_Threshold                                    float64
PPI_Likelihood                                   float64
HFIAS_status                                    category
score_HDDS_GoodSeason          

In [112]:
market_data, pred_market = fun.drop_response_rows_with_NAs(clean, "Market_Orientation", "PPI_Likelihood") #2


In [94]:
# market_data = fun.delete_id_columns(clean) #1
# market_data, pred_market = fun.drop_response_rows_with_NAs(market_data, "Market_Orientation", "PPI_Likelihood") #2
# market_data = fun.replace_NAN_with_na(market_data) #3
# market_data = fun.entry_to_lowercase(market_data) #4
# market_data = fun.remove_underscores_spaces(market_data) #5
# market_data = fun.convert_to_categorical(market_data) #6
# market_data = fun.impute_data(market_data)

In [113]:

market_data.YEAR = market_data.YEAR.astype('category')

In [114]:
def standarize_data(df):
    
    """
    Input: a dataset
    action: returns numeric column values scaled by mean and standard deviation
    """
    numeric_data = df.select_dtypes(include=['float64', 'int64'])
    for i in numeric_data.columns:
        df[i] = (df[i] - df[i].mean())/df[i].std()
    return df

In [115]:
market_data = standarize_data(market_data)

In [98]:
market_data.dtypes

crop_count                                       float64
crop_name_1                                     category
crop_harvest_1                                  category
crop_intercrop_1                                category
YEAR                                            category
Country                                         category
HHsizemembers                                    float64
HHsizeMAE                                        float64
HouseholdType                                   category
Head_EducationLevel                             category
LandOwned                                        float64
LandCultivated                                   float64
LivestockHoldings                                float64
NrofMonthsFoodInsecure                           float64
HFIAS_status                                    category
score_HDDS_GoodSeason                            float64
score_HDDS_farmbasedGoodSeason                   float64
score_HDDS_purchasedGoodSeason 

## Ridge Regression y = Market orientation

In [116]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### 1. create dummy

In [117]:
def get_dummyXs_y(df, y_var):
    
    y = df[y_var]
    X  = df.drop(y_var, axis = 1)
    X_cat = X.select_dtypes(include = ["category", "O"])
    X_num = X.select_dtypes(include=['float64', 'int64'])
    
    X_cat_dummy = pd.get_dummies(X_cat)
    newX = pd.concat([X_num, X_cat_dummy], axis = 1)
    
    return newX, y

In [118]:
X_market_ridge = get_dummyXs_y(market_data, "Market_Orientation")[0]
y_market_ridge = get_dummyXs_y(market_data, "Market_Orientation")[1]

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X_market_ridge,y_market_ridge, test_size = 0.3, random_state = 2021)


In [120]:
#ridge_cv=RidgeCV(alphas=np.arange(2,,0.001))
ridge_cv=RidgeCV(alphas=np.arange(0.01,2,0.001))
model_cv=ridge_cv.fit(X_train,y_train)
model_cv.alpha_

1.9989999999999983

In [121]:
#refit model with alpha =3 and get score
rir = Ridge(alpha = model_cv.alpha_)
rir.fit(X_train,y_train)
y_pred_ridge = rir.predict(X_test)
MSE_ridge_market = mean_squared_error(y_test,y_pred_ridge)
MSE_ridge_market

3.472907853427353

In [86]:
X_train.columns

Index(['crop_count', 'YEAR', 'HHsizemembers', 'HHsizeMAE', 'LandOwned',
       'LandCultivated', 'LivestockHoldings', 'NrofMonthsFoodInsecure',
       'score_HDDS_GoodSeason', 'score_HDDS_farmbasedGoodSeason',
       ...
       'Head_EducationLevel_religiousschool', 'Head_EducationLevel_secondary',
       'HFIAS_status_foodsecure', 'HFIAS_status_mildlyfi',
       'HFIAS_status_moderatelyfi', 'HFIAS_status_severelyfi',
       'continent_africa', 'continent_asia', 'continent_centralamerica',
       'continent_southamerica'],
      dtype='object', length=179)

In [ ]:
# -2.17617050e-02,  1.57421573e+00, -1.55559476e+00, -4.08594181e-03

In [122]:
rir.coef_

array([-3.49784197e-03, -3.22714077e-02, -2.08077846e-03, -2.64679825e-02,
        1.05367239e-03, -1.19704049e-01, -8.21888676e-03, -8.74441620e-03,
       -4.24932490e-03,  4.61104511e-03, -2.36644066e-02, -3.06893995e-02,
       -8.78657268e-03, -4.90293894e-04,  1.59865145e-03, -3.94747070e-03,
        5.83052767e-03,  6.20580428e-05, -7.79237893e-04, -3.33712261e-03,
       -1.78383855e-01,  8.42325442e-01,  1.78816795e-01, -1.85396393e-01,
       -2.44940023e-01,  3.51245421e-02, -3.34531603e-02, -3.12013951e-03,
       -4.81399355e-02,  4.81399355e-02, -6.74179288e-01,  0.00000000e+00,
        1.60876041e-01, -4.42052032e-02, -1.04919116e-01,  3.96509954e-02,
        7.77205006e-01, -2.54875961e-01,  3.64929698e-02, -3.44839779e-02,
        1.04013114e-01,  1.45212458e-01,  9.22275184e-02, -2.86931193e-01,
        0.00000000e+00, -1.42941597e-01,  5.08397957e-03,  0.00000000e+00,
        2.87098975e-01, -7.60606006e-02,  1.87659166e-01, -1.44394237e-01,
       -1.07208981e-01,  

In [123]:
feature_imp  = pd.DataFrame([X_train.columns, rir.coef_]).T

In [124]:
feature_imp = feature_imp.sort_values(by = 1,ascending = False).reset_index(drop = True)
feature_imp.iloc[0:15,:]

,0,1
0,livestock_prodsales_USD_PPP_pHH_Yr,0.842325
1,crop_name_1_broadbeans,0.777205
2,Country_india,0.502535
3,HouseholdType_nonparenthead,0.473117
4,Head_EducationLevel_none,0.459644
5,Country_mali,0.431082
6,crop_name_1_tomato,0.428392
7,crop_name_1_sweetpotato,0.425576
8,HouseholdType_together,0.422399
9,Country_zambia,0.373268


## Ridge regression with y=PPI_Likelihood

In [125]:
clean = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/clean.csv")

In [126]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [127]:

clean  = fun.delete_id_columns(clean) 
clean = fun.replace_NAN_with_na(clean)
clean = fun.entry_to_lowercase(clean)
clean = fun.remove_underscores_spaces(clean)
clean = fun.impute_data(clean)
clean = fun.convert_to_categorical(clean)
clean

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,YEAR,Country,HHsizemembers,HHsizeMAE,HouseholdType,Head_EducationLevel,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,2.0,groundnut,na,na,2015,tanzania,4,3.32,single,primary,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,2.0,millet,na,na,2015,tanzania,10,5.85,together,primary,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,2.0,groundnut,na,na,2015,tanzania,5,4.32,together,illiterate,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,2.0,cowpea,na,na,2015,tanzania,11,9.33,together,primary,...,0.000000,0.000000,1.000000,0.310907,257.234727,257.234727,0,0.999984,0.000016,africa
4,2.0,sesame,na,na,2015,tanzania,6,5.41,together,secondary,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11957,3.0,maize,goodharvest,monoculture,2018,ghana,8,5.43,couple,primary,...,655.292919,0.000000,1.000000,0.779661,1615.869392,339.743848,7,0.629176,0.370824,africa
11958,5.0,maize,goodharvest,monoculture,2018,ghana,7,4.78,couple,noschool,...,0.000000,0.000000,1.000000,0.310907,355.648536,355.648536,5,1.000000,0.000000,africa
11959,2.0,maize,badharvest,intercrop,2018,ghana,4,3.20,mansingle,noschool,...,49.859244,0.000000,0.456522,0.456522,379.050204,276.284247,0,0.000000,1.000000,africa
11960,2.0,maize,badharvest,intercrop,2018,ghana,4,3.10,couple,noschool,...,0.000000,0.000000,1.000000,0.310907,693.504198,693.504198,0,1.000000,0.000000,africa


In [128]:
ppi_data, pred_market = fun.drop_response_rows_with_NAs(clean,"PPI_Likelihood" ,"Market_Orientation") #2


In [153]:
# ppi_data = fun.delete_id_columns(clean) #1
# ppi_data, pred_market = fun.drop_response_rows_with_NAs(ppi_data,"PPI_Likelihood" ,"Market_Orientation") #2
# ppi_data = fun.replace_NAN_with_na(ppi_data) #3
# ppi_data = fun.entry_to_lowercase(ppi_data) #4
# ppi_data = fun.remove_underscores_spaces(ppi_data) #5
# ppi_data = fun.convert_to_categorical(ppi_data) #6
# ppi_data = fun.impute_data(ppi_data)#7

In [129]:
ppi_data.YEAR = ppi_data.YEAR.astype('category')
ppi_data = standarize_data(ppi_data)


In [130]:
X_ppi = get_dummyXs_y(ppi_data, "PPI_Likelihood")[0]
y_ppi = get_dummyXs_y(ppi_data, "PPI_Likelihood")[1]

In [131]:
X_train_ppi, X_test_ppi, y_train_ppi, y_test_ppi = train_test_split(X_ppi ,y_ppi, test_size = 0.3, random_state = 2021)


In [132]:


ppi_ridge_cv=RidgeCV(alphas=np.arange(0.01,2,0.001))
ppi_model_cv=ppi_ridge_cv.fit(X_train_ppi,y_train_ppi)
ppi_model_cv.alpha_

1.9989999999999983

In [133]:
#refit model with alpha =3 and get score
ridge_ppi = Ridge(alpha = ppi_model_cv.alpha_)
ridge_ppi.fit(X_train_ppi,y_train_ppi)
y_pred_ridge_ppi = ridge_ppi.predict(X_test_ppi)
MSE_ppi = np.mean((y_pred_ridge_ppi-np.array(y_test_ppi))**2)
MSE_ppi

0.6870890680656784

In [137]:
#ridge_ppi.coef_

In [135]:
feature_imp_ppi  = pd.DataFrame([X_train.columns, ridge_ppi.coef_]).T
feature_imp_ppi = feature_imp_ppi.sort_values(by = 1,ascending = False).reset_index(drop = True)
feature_imp_ppi.iloc[0:15,:]

,0,1
0,Country_zambia,1.50551
1,Country_mali,1.48262
2,Country_elsalvador,1.17489
3,Country_malawi,0.948146
4,crop_name_1_papaya,0.579726
5,crop_name_1_zucchini,0.540962
6,HouseholdType_polygamous,0.475944
7,crop_name_1_peas,0.464779
8,crop_name_1_rambutan,0.459123
9,crop_name_1_hybiscus,0.416353


In [136]:
pd.concat([feature_imp_ppi, feature_imp],axis = 1).iloc[0:20,:]

,0,1,0,1
0,Country_zambia,1.50551,livestock_prodsales_USD_PPP_pHH_Yr,0.842325
1,Country_mali,1.48262,crop_name_1_broadbeans,0.777205
2,Country_elsalvador,1.17489,Country_india,0.502535
3,Country_malawi,0.948146,HouseholdType_nonparenthead,0.473117
4,crop_name_1_papaya,0.579726,Head_EducationLevel_none,0.459644
5,crop_name_1_zucchini,0.540962,Country_mali,0.431082
6,HouseholdType_polygamous,0.475944,crop_name_1_tomato,0.428392
7,crop_name_1_peas,0.464779,crop_name_1_sweetpotato,0.425576
8,crop_name_1_rambutan,0.459123,HouseholdType_together,0.422399
9,crop_name_1_hybiscus,0.416353,Country_zambia,0.373268


In [85]:
clean.Head_EducationLevel.unique()

[primary, illiterate, secondary, literate, noschool, ..., postsecondary, informal, none, religiousschool, islamicschool]
Length: 27
Categories (27, object): [primary, illiterate, secondary, literate, ..., informal, none, religiousschool, islamicschool]

In [172]:
clean.Head_EducationLevel.isna().sum()

412

## drop countries 

In [153]:
clean = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/clean.csv")
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

clean  = fun.delete_id_columns(clean) 
clean = fun.replace_NAN_with_na(clean)
clean = fun.entry_to_lowercase(clean)
clean = fun.remove_underscores_spaces(clean)
clean = fun.impute_data(clean)
clean = fun.convert_to_categorical(clean)
clean

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,YEAR,Country,HHsizemembers,HHsizeMAE,HouseholdType,Head_EducationLevel,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,2.0,groundnut,na,na,2015,tanzania,4,3.32,single,primary,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,2.0,millet,na,na,2015,tanzania,10,5.85,together,primary,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,2.0,groundnut,na,na,2015,tanzania,5,4.32,together,illiterate,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,2.0,cowpea,na,na,2015,tanzania,11,9.33,together,primary,...,0.000000,0.000000,1.000000,0.310907,257.234727,257.234727,0,0.999984,0.000016,africa
4,2.0,sesame,na,na,2015,tanzania,6,5.41,together,secondary,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11957,3.0,maize,goodharvest,monoculture,2018,ghana,8,5.43,couple,primary,...,655.292919,0.000000,1.000000,0.779661,1615.869392,339.743848,7,0.629176,0.370824,africa
11958,5.0,maize,goodharvest,monoculture,2018,ghana,7,4.78,couple,noschool,...,0.000000,0.000000,1.000000,0.310907,355.648536,355.648536,5,1.000000,0.000000,africa
11959,2.0,maize,badharvest,intercrop,2018,ghana,4,3.20,mansingle,noschool,...,49.859244,0.000000,0.456522,0.456522,379.050204,276.284247,0,0.000000,1.000000,africa
11960,2.0,maize,badharvest,intercrop,2018,ghana,4,3.10,couple,noschool,...,0.000000,0.000000,1.000000,0.310907,693.504198,693.504198,0,1.000000,0.000000,africa


In [154]:
test1  = clean.drop("continent", axis =1)
test1.head()

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,YEAR,Country,HHsizemembers,HHsizeMAE,HouseholdType,Head_EducationLevel,...,livestock_prodsales_USD_PPP_pHH_Yr,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl
0,2.0,groundnut,na,na,2015,tanzania,4,3.32,single,primary,...,38.500640,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000
1,2.0,millet,na,na,2015,tanzania,10,5.85,together,primary,...,0.000000,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001
2,2.0,groundnut,na,na,2015,tanzania,5,4.32,together,illiterate,...,128.335467,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000
3,2.0,cowpea,na,na,2015,tanzania,11,9.33,together,primary,...,0.000000,0.000000,0.000000,1.000000,0.310907,257.234727,257.234727,0,0.999984,0.000016
4,2.0,sesame,na,na,2015,tanzania,6,5.41,together,secondary,...,210.892267,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000


In [155]:
p1, predm = fun.drop_response_rows_with_NAs(test1,"PPI_Likelihood" ,"Market_Orientation") #2



In [156]:
p1.YEAR =p1.YEAR.astype('category')
p1 = standarize_data(p1)
X_ppi = get_dummyXs_y(p1, "PPI_Likelihood")[0]
y_ppi = get_dummyXs_y(p1, "PPI_Likelihood")[1]

In [157]:
X_train_ppi, X_test_ppi, y_train_ppi, y_test_ppi = train_test_split(X_ppi ,y_ppi, test_size = 0.3, random_state = 2021)


ppi_ridge_cv=RidgeCV(alphas=np.arange(0.01,2,0.001))
ppi_model_cv=ppi_ridge_cv.fit(X_train_ppi,y_train_ppi)
ppi_model_cv.alpha_

1.9989999999999983

In [158]:
#refit model with alpha =3 and get score
r1 = Ridge(alpha = ppi_model_cv.alpha_)
r1.fit(X_train_ppi,y_train_ppi)
y_pred_ridge_ppi = r1.predict(X_test_ppi)
MSE_ppi = np.mean((y_pred_ridge_ppi-np.array(y_test_ppi))**2)
MSE_ppi

0.6863987409320556

In [159]:
feature_imp_ppi  = pd.DataFrame([X_train_ppi.columns, r1.coef_]).T
feature_imp_ppi = feature_imp_ppi.sort_values(by = 1,ascending = False).reset_index(drop = True)
feature_imp_ppi.iloc[0:15,:]

,0,1
0,Country_zambia,1.71073
1,Country_mali,1.68728
2,Country_malawi,1.14805
3,Country_elsalvador,0.994282
4,crop_name_1_papaya,0.581091
5,crop_name_1_zucchini,0.540053
6,HouseholdType_polygamous,0.482001
7,crop_name_1_peas,0.46471
8,crop_name_1_rambutan,0.460488
9,crop_name_1_hybiscus,0.417097
